In [2]:
import requests

In [3]:
### Attention, change this Task ID to your job!

GNPS_Deconvolution_Task = "adf31b8a6f2445c3addc60ceca3f8424"

In [9]:
# Downloading Data. Data will appear in specs_ms.mgf file

mgf_url = "https://gnps.ucsd.edu/ProteoSAFe/DownloadResultFile?task={}&block=main&file=clustered_mgf/".format(GNPS_Deconvolution_Task)
r = requests.get(mgf_url)
r.raise_for_status()

with open("specs_ms.mgf", "w") as out_file:
    out_file.write(r.text)

In [ ]:
# Do stuff with it

# Lorem Ipsum